In [7]:
import numpy as np
import random
import warnings
import random
import copy
import pandas as pd
import time
from datetime import datetime
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from sympy import symbols, sympify, lambdify

# Configuración warnings
################################################################################
import warnings
warnings.filterwarnings('ignore')

In [8]:
doc2 = open("C:/Users/antoc/Downloads/PortableGit/practica2024/hub_data.txt")
lineas2 = doc2.readlines()

H_0= 70

# Divide cada línea en columnas usando espacio como separador
datos_separados2 = [linea.strip().split(' ') for linea in lineas2]

datos= []
for fila in datos_separados2:
    datos.append(fila)

z = []
for i in range(len(datos)):
      a = float(datos[i][0])
      z.append(a)
z =np.array(z)

H = []
for i in range(len(datos)):
   if datos[i][1] == '':
      a = float(datos[i][2])
      H.append(a)
   else:
      a = float(datos[i][1])
      H.append(a)
H = np.array(H)

desv = []
for i in range(len(datos)):
    a = float(datos[i][2])
    desv.append(a)


### Clase individuo

In [9]:
class Individuo:
    x = symbols('x')
    
    def __init__(self, grammar, operaciones):

        self.elementos = []
        self.string = []
        self.func = None
        # Fitness del individuo
        self.fitness = None

        #Se genera un valor aleatorio para las variables
        self.elementos.append(random.choice(grammar))
        self.elementos.append(random.choice(operaciones))
        self.elementos.append(random.choice(grammar))
        self.elementos.append(random.choice(operaciones))
        self.elementos.append(random.choice(grammar))
        self.elementos.append(random.choice(grammar))
        self.elementos.append(random.choice(operaciones))
        self.elementos.append(random.choice(grammar))


    @staticmethod
    def polix(x):
        return x**x

    @staticmethod
    def poli1(x):
        return (x + 1)

    @staticmethod
    def poli2(x):
        return x**2

    @staticmethod
    def poli3(x):
        return x**3
    # Crear lista con coeficientes
    coeficientes = [random.random() for i in range(5)]
    # Crear lista de funciones constantes
    funciones_c = [lambda x, i=i: Individuo.coeficientes[i] for i in range(5)]
        
    #x deben estar entre paréntesis para no confundir con otros x en el string
    @staticmethod
    def a_string(funcion):
        if funcion == Individuo.polix:
            return '(x)**(x)'
        elif funcion == Individuo.poli1:
            return '((x)+1)'
        elif funcion == Individuo.poli2:
            return '(x)**2'
        elif funcion == Individuo.poli3:
            return '(x)**3'
        for i in range(len(Individuo.coeficientes)):
            if funcion == Individuo.funciones_c[i]:
                    return str(Individuo.coeficientes[i])

    @staticmethod
    def pasar_a_funcion(expresion):
        a_sympy = sympify(expresion)
        a_python = lambdify(Individuo.x, a_sympy, 'numpy')
        return a_python

    @staticmethod
    def composicion(lista):
        modi_int = Individuo.a_string(lista[-1])
        for i in range(2, len(lista) + 1):
            modi_ext = Individuo.a_string(lista[-i])
            modi_int = modi_ext.replace('(x)', '(' + modi_int + ')')
        return modi_int

    def convertir_lista_a_str(self):
        for i in range(len(self.elementos)):
            if callable(self.elementos[i]):
                if i != 0 and callable(self.elementos[i - 1]):
                    self.string = self.string
                elif i < len(self.elementos) - 1 and callable(self.elementos[i + 1]):
                    compo = []
                    for j in range(i, len(self.elementos)):
                        if callable(self.elementos[j]):
                            compo.append(self.elementos[j])

                    self.string += Individuo.composicion(compo)
                else:
                    self.string += Individuo.a_string(self.elementos[i])
            else:
                self.string += self.elementos[i]
        self.string = ''.join(self.string)

    def convertir_lista_a_funcion(self):
        self.func = Individuo.pasar_a_funcion(self.string)
    

    def calcular_fitness(self):
        self.fitness = np.square((H - self.func)/desv)
    

    def mutar(self, grammar, operaciones, tasa_mutacion= 0.05):
        for i in range(1, len(self.elementos)):
            # Genera un número aleatorio entre 0 y 1
            probabilidad = random.random()
            # Verifica si el número aleatorio es menor que la probabilidad deseada
            if probabilidad < tasa_mutacion:
                if callable(self.elementos):
                    self.elementos[i] = random.choice(grammar)
                else:
                    self.elementos[i] = random.choice(operaciones)

          # REINICIO DEL VALOR Y DEL FITNESS
        #-----------------------------------------------------------------------
        # Dado que el individuo ha mutado, el valor de su fitness y de la
        # función objetivo ya no son validos.
        self.fit = None
        self.valor_funcion = None
        
        
    


In [10]:
grammar = [Individuo.polix, Individuo.poli1, Individuo.poli2, Individuo.poli3] + Individuo.funciones_c

operaciones = ['+', '-', '*', '/']

indiv =Individuo(grammar, operaciones)
indiv.convertir_lista_a_str()

In [11]:
print(indiv.elementos)
print(indiv.string)
indiv.convertir_lista_a_funcion()
indiv.mutar(grammar, operaciones)

[<function Individuo.<listcomp>.<lambda> at 0x0000020E05A92F70>, '/', <function Individuo.poli1 at 0x0000020E05A92310>, '*', <function Individuo.<listcomp>.<lambda> at 0x0000020E05A92DC0>, <function Individuo.poli3 at 0x0000020E05A92C10>, '/', <function Individuo.<listcomp>.<lambda> at 0x0000020E05A92DC0>]
0.3201201495481564/((x)+1)*0.9639672319280824/0.9639672319280824


### Clase población

In [12]:
class Poblacion:      
    def __init__(self, n_variables, n_individuos):
        
        self.n_variables = n_variables
        self.n_individuos = n_individuos
        # Lista de los individuos de la población
        self.individuos = []
        # Etiqueta para saber si la población ha sido optimizada
        self.optimizado = False
        # Número de iteraciones de optimización llevadas a cabo
        self.iter_optimizacion = None
        # Mejor individuo de la población
        self.mejor_individuo = None
        # Fitness del mejor individuo de la población (el de mayor fitness)
        self.mejor_fitness = None
        # Valor de las variables del mejor individuo de la población
        self.mejor_valor_variables = None
        # Información de todas los individuos de la población en cada generación
        self.historico_individuos = []
        # Valor de las variables del mejor individuo en cada generación
        self.historico_mejor_valor_variables = []
        # Fitness del mejor individuo en cada generación
        self.historico_mejor_fitness = []
        # Valor de la función objetivo del mejor individuo en cada generación
        self.historico_mejor_valor_funcion = []
        # Diferencia absoluta entre el mejor fitness de generaciones consecutivas
        self.diferencia_abs = []
        # data.frame con la información del mejor fitness y valor de variables
        # encontrado en cada generación, así como la diferencia respecto a la 
        # generación anterior.
        self.resultados_df = None
        # Fitness del mejor individuo de todas las generaciones
        self.fitness_optimo = None
        # Valor de las variables del mejor individuo de todas las generaciones
        self.valor_variables_optimo = None
        # Valor de función objetivo del mejor individuo de todas las generaciones
        self.valor_funcion_optimo = None

        # SE CREAN LOS INDIVIDUOS DE LA POBLACIÓN Y SE ALMACENAN
        # ----------------------------------------------------------------------
        for i in np.arange(n_individuos):
            individuo_i = Individuo(n_variables = self.n_variables,)
            self.individuos.append(individuo_i)

    def mostrar_individuos(self, n=None):
        if n is None:
            n = self.n_individuos
        elif n > self.n_individuos:
            n = self.n_individuos

        for i in np.arange(n):
            print(self.individuos[i])
        return(None)
    

    def evaluar_poblacion(self):
        # SE EVALÚA CADA INDIVIDUO DE LA POBLACIÓN
        # ----------------------------------------------------------------------
        for i in np.arange(self.n_individuos):
            self.individuos[i].calcular_fitness()

        # MEJOR INDIVIDUO DE LA POBLACIÓN
        # ----------------------------------------------------------------------
        # Se identifica el mejor individuo de toda el población, el de mayor fitness.

        # Se selecciona inicialmente como mejor individuo el primero.
        self.mejor_individuo = copy.deepcopy(self.individuos[0])
        # Se comparan todas los individuos de la población.
        for i in np.arange(self.n_individuos):
            if self.individuos[i].fitness > self.mejor_individuo.fitness:
                self.mejor_individuo = copy.deepcopy(self.individuos[i])

        # Se extrae la información del mejor individuo de la población.
        self.mejor_fitness = self.mejor_individuo.fitness
        self.mejor_valor_variables = self.mejor_individuo.elementos

    def cruzar_individuos(self, parental_1, parental_2):
        # CREACIÓN DE LA DESCENDENCIA
        # ----------------------------------------------------------------------
        # Se extraen los parentales acorde a los índices indicados.
        parental_1 = self.individuos[parental_1]
        parental_2 = self.individuos[parental_2]
        
        # Se clona uno de los parentales para utilizarlo como plantilla del nuevo individuo.
        descendencia_copy = copy.deepcopy(parental_1)
        descendencia = [item for item in descendencia_copy if callable(item)]
        descendencia.elementos = np.repeat(None, descendencia.n_variables())
        descendencia.fitness = None

        # Se seleccionan aleatoriamente las posiciones que se heredan del
        # parental_1 y del parental 2.
        herencia_parent_1 = np.random.choice(
                                a       = [True, False],
                                size    = descendencia.n_variables(),
                                p       = [0.5, 0.5],
                                replace = True
                            )
        herencia_parent_2 = np.logical_not(herencia_parent_1)

        # Se transfieren los valores al nuevo individuo.
        descendencia.valor_variables[herencia_parent_1] = parental_1.valor_variables[herencia_parent_1]
        descendencia.valor_variables[herencia_parent_2] = parental_2.valor_variables[herencia_parent_2]
        
        # Se crea un deepcopy para que el nuevo individuo sea independiente de 
        # los parentales. Esto evita problemas si posteriormente se muta.
        descendencia = copy.deepcopy(descendencia)
    
    # Se utiliza el método de selección "tournament"
    #n : número de individuos de la población seleccionados.
    def seleccionar_individuo(self, n, return_indices=True):
        # SELECCIÓN DE INDIVIDUOS
        # ----------------------------------------------------------------------
        # Se crea un array con el fitness de cada individuo de la población.
        array_fitness = np.repeat(None, self.n_individuos)
        for i in np.arange(self.n_individuos):
            array_fitness[i] = copy.copy(self.individuos[i].fitness)
        
        ind_seleccionado = np.repeat(None,n)
        for i in np.arange(n):
            # Se seleccionan aleatoriamente dos parejas de individuos.
            candidatos_a = np.random.choice(
                            a       = np.arange(self.n_individuos),
                            size    = 2,
                            replace = False
                        )
            candidatos_b = np.random.choice(
                            a       = np.arange(self.n_individuos),
                            size    = 2,
                            replace = False
                        )
            # De cada pareja se selecciona el de mayor fitness.
            if array_fitness[candidatos_a[0]] > array_fitness[candidatos_a[1]]:
                ganador_a = candidatos_a[0]
            else:
                ganador_a = candidatos_a[1]

            if array_fitness[candidatos_b[0]] > array_fitness[candidatos_b[1]]:
                ganador_b = candidatos_b[0]
            else:
                ganador_b = candidatos_b[1]

            # Se comparan los dos ganadores de cada pareja.
            if array_fitness[ganador_a] > array_fitness[ganador_b]:
                ind_final = ganador_a
            else:
                ind_final = ganador_b
            
            ind_seleccionado[i] = ind_final

    def crear_nueva_generecion(self, grammar, operaciones, tasa_mutacion, elitismo=0.1, prob_mut=0.01):
        # Lista donde almacenar los individuos de la nueva generación.
        nuevos_individuos = []

        # ELITISMO
        # ----------------------------------------------------------------------
        if elitismo > 0:
            # Número de individuos que pasan directamente a la siguiente
            # generación.
            n_elitismo = int(np.ceil(self.n_individuos*elitismo))

            # Se identifican los n_elitismo individuos con mayor fitness (élite).
            array_fitness = np.repeat(None, self.n_individuos)
            for i in np.arange(self.n_individuos):
                array_fitness[i] = copy.copy(self.individuos[i].fitness)
            rank = np.flip(np.argsort(array_fitness))
            elite = [copy.deepcopy(self.individuos[i]) for i in rank[:n_elitismo]]
            # Se añaden los individuos élite a la lista de nuevos individuos.
            nuevos_individuos = nuevos_individuos + elite
        else:
            n_elitismo = 0
            
        # CREACIÓN DE NUEVOS INDIVIDUOS POR CRUCES
        # ----------------------------------------------------------------------
        for i in np.arange(self.n_individuos-n_elitismo):
            # Seleccionar parentales
            indice_parentales = self.seleccionar_individuo(
                                    n                = 2,
                                    return_indices   = True,
                                 )
            # Cruzar parentales para obtener la descendencia
            descendencia = self.cruzar_individuos(
                            parental_1 = indice_parentales[0],
                            parental_2 = indice_parentales[1],
                           )
            # Mutar la descendencia
            descendencia.mutar(
                grammar = grammar,
                operaciones = operaciones, 
                tasa_mutacion= tasa_mutacion
            )
            # Se añade la descendencia a la lista de nuevos individuos. Para
            # que no de error la unión, se introduce el individuo descendencia
            # dentro de una lista.
            nuevos_individuos = nuevos_individuos + [descendencia]

        # ACTUALIZACIÓN INFORMACIÓN DE LA POBLACIÓN
        # ----------------------------------------------------------------------
        self.individuos = copy.deepcopy(nuevos_individuos)
        self.mejor_individuo = None
        self.mejor_fitness = None
        self.mejor_valor_variables = None
        
    def optimizar(self. n_generaciones = 50,
                  elitismo=0.1, tasa_mutacion=0.01,
                  parada_temprana=False, rondas_parada=None,
                  tolerancia_parada=None):
        

        
